# ロジスティックシグモイド関数

## モデル化
$$
p\left( { y = 1 } | { \boldsymbol {\mathrm {x}} } \right) \approx f\left( u \right)\\
p\left( { y = 0 } | { \boldsymbol {\mathrm {x}} } \right) \approx 1 - f\left( u \right)
$$

### 活性化関数 : ロジスティックシグモイド関数
$$
f\left( u \right) = \frac {1}{1 + e^{-u}}\\
u = \boldsymbol {\mathrm {w}}^{T} \boldsymbol {\mathrm {x}} + \boldsymbol {\mathrm {b}}
$$

### 出力層の活性化関数 : ロジスティックシグモイド関数


### 誤差関数 : 交差エントロピー(負の対数尤度関数)
$$
L\left( \boldsymbol {\mathrm {w}} ; \boldsymbol {\mathrm {x}} \right) = \prod _{i=1}^{N}{p\left( { y_{i} } | { \boldsymbol {\mathrm {x}} ; \boldsymbol {\mathrm {w}}} \right)} = \prod _{i=1}^{N}{\left\{ f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right\}^{y_{i}} \left\{ 1 - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right\}^{1 - y_{i}} }\\
\log {L\left( \boldsymbol {\mathrm {w}} ; \boldsymbol {\mathrm {x}} \right)} = \sum _{i=1}^{N}{\left\{ y_{i} \log {f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right)} + (1 - y_{i}) \log {\left[ 1 - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right]} \right\}}
$$
上記の対数尤度関数の符号を反転した関数を誤差関数とし、この値の最小化を考える。
$$
E\left( \boldsymbol {\mathrm {w}} \right) = - \sum _{i=1}^{N}{\left\{ y_{i} \log {f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right)} + (1 - y_{i}) \log {\left[ 1 - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right]} \right\}}
$$
$$
\boldsymbol {\mathrm {w}} = arg \min _{ \boldsymbol {\mathrm {w}} }{E\left( \boldsymbol {\mathrm {w}} \right)}
$$

### 学習法 : 
$$
\nabla E \equiv \frac {\partial E}{\partial \boldsymbol {w}} = \left( \frac {\partial E}{\partial w_{1}}, \dots, \frac {\partial E}{\partial w_{p}} \right)^{T}\\
\boldsymbol {w}^{(t+1)} = \boldsymbol {w}^{(t)} - \epsilon \nabla E
$$

勾配ベクトルの各要素は次式になる。
$$
\frac {\partial E}{\partial w_{j}} = - \sum _{i=1}^{N}{\left[ y_{i} - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right] x_{j}}
$$

#### (バッチ)勾配降下法(gradient descent)
$$
w_{j}^{(t+1)} = w_{j}^{(t)} + \epsilon \sum _{i=1}^{N}{\left[ y_{i} - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right] x_{j}}
$$

#### 確率的勾配降下法(stochastic gradient descent)
$$
w_{j}^{(t+1)} = w_{j}^{(t)} + \epsilon \left[ y_{i} - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right] x_{j}
$$

 - ランダムに1つの訓練データ$i$を用いてパラメータを更新
 - 収束するまで、$N$個のデータ全体に対してランダムに訓練データを抽出し、繰り返し学習を行う

```python
for epoch in range(epochs):
    shuffle(data)   # エポック(反復)ごとにデータをシャッフル
    for datum in data:    # データ1個ずつでパラメータを更新
        params_grad = evaluate_gradient(error_function, params, dataum)
        params -= learning_rate * params_grad
```

#### ミニバッチ勾配降下法(minibatch gradient descent)

 - $D_{t}$ : 1つのミニバッチ$t$
 - $N_{t}$: ミニバッチ$t$が含む訓練データ

$$
w_{j}^{(t+1)} = w_{j}^{(t)} + \epsilon \sum _{i \in D_{i}}{\left[ y_{i} - f\left( \boldsymbol {\mathrm {x}}_{i} ; \boldsymbol {\mathrm {w}} \right) \right] x_{j}}
$$

```python
for epoch in range(epochs):
    shuffle(data)
    batches = get_batches(data, batch_size=M)
    for batch in batches:    # ミニバッチごとにパラメータを更新
        params_grad = evaluate_gradient(error_function, params, batch)
        params -= learning_rate * params_grad
```

# 実装
## TensorFlowによる実装

In [5]:
import numpy as np
import tensorflow as tf

### モデルの定義

 - パラメータの定義
 - 出力層の定義
 - 誤差関数の定義
 - 最適化手法の定義

In [19]:
# 重み(weight), バイアス(bias)
w = tf.Variable(tf.zeros([2, 1]))
b = tf.Variable(tf.zeros([1]))


x = tf.placeholder(tf.float32, shape=[None, 2])    # 特徴ベクトル
y = tf.placeholder(tf.float32, shape=[None, 1])    # 正解データ
f = tf.nn.sigmoid(tf.matmul(x, w) + b)    # 出力層 : シグモイド関数

# 誤差関数
cross_entropy = - tf.reduce_sum(y * tf.log(f) + (1 - y) * tf.log(1 - f))

# 学習法
LEARNING_RATE = 0.1
train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)

# 予測
correct_prediction = tf.equal(
        tf.to_float(tf.greater(f, 0.5)),
        y
    )

### データセットの用意

In [20]:
# 学習用データ
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

### セッションの初期化

In [21]:
# ---セッション---

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)    # ここではじめてモデルの定義で宣言した変数・式の初期化が行われる

### 学習

In [22]:
# 学習
for epoch in range(200):
    # 勾配降下法の実行
    sess.run(
        train_step, 
        feed_dict={x: X, y: Y}
    )

### 評価

In [23]:
classified = correct_prediction.eval(session=sess, feed_dict={x: X, y: Y})   # 評価
prob = f.eval(session=sess, feed_dict={x: X})    # 学習済みのモデルで予測

print('===[classified]===')
print('{0}'.format(classified))
print('===[prob]=== ')
print('{0}'.format(prob))

===[classified]===
[[ True]
 [ True]
 [ True]
 [ True]]
===[prob]=== 
[[0.22355042]
 [0.9142595 ]
 [0.9142595 ]
 [0.99747413]]


## Kerasによる実装
### モデルの定義

 - 出力層の定義
 - 最適化手法の定義

In [25]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD    # 確率的勾配降下法

# 出力層の定義
model = Sequential([
        Dense(input_dim=2, units=1),    # 層
        Activation('sigmoid')                    # 活性化関数
    ])

Using TensorFlow backend.


 - `Sequential()` : 層構造のモデルを定義するためのメソッド. ここに実際の層を入れ込むことで、モデルを設定する.
     - `Dense(input_dim=2, units=1)` : 入力の次元2、出力の次元1のネットワーク構造を持つ層を生成. これは次式における以下に相当する層を作ったことになる.
     $$
     w_{1} x_{1} + w_{2} x_{2} + b
     $$
     - `Activation('sigmoid')` : ニューロンの出力を表現. ここに活性化関数を指定. 次式に相当する層を作る.
     $$
     y = \frac {1}{1 + \mathrm {exp}\left( w_{1} x_{1} + w_{2} x_{2} + b \right)}
     $$

これでモデルの出力部分までは定義した。また、`Sequential()`だけを事前に宣言しておき、後から`model.add()`でどんどん層を追加していく書き方も可能。
```python
model = Sequential()
model.add(Dense(input_dim=2, units=1))
model.add(Activation('sigmoid'))
```

In [26]:
# 最適化手法の定義 : 確率的勾配降下法
LEARNING_RATE = 0.1
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=LEARNING_RATE))

### データセットの用意

In [27]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

### 学習

In [28]:
model.fit(X, Y, epochs=200, batch_size=1)
# epochs : エポック数. 最大更新回数
# batch_size : (ミニ)バッチの大きさ

Epoch 1/200
4/4 [==============================] - 0s 53ms/step - loss: 0.4881
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4642
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4449
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 0.4292
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4163
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4056
Epoch 7/200
4/4 [==============================] - 0s 2ms/step - loss: 0.3960
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.3878
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.3804
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.3737
Epoch 11/200
4/4 [==============================] - 0s 2ms/step - loss: 0.3678
Epoch 12/200
4/4 [==============================] - 0s 3ms/step - loss: 0.3622
Epoch 13/200
4/4 [==============================] - 0s 6ms/s

4/4 [==============================] - 0s 2ms/step - loss: 0.1670
Epoch 105/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1660
Epoch 106/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1650
Epoch 107/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1640
Epoch 108/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1631
Epoch 109/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1621
Epoch 110/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1611
Epoch 111/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1602
Epoch 112/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1593
Epoch 113/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1583
Epoch 114/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1574
Epoch 115/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1565
Epoch 116/200
4/4 [==============================] - 0

### 評価

In [30]:
classes = model.predict_classes(X, batch_size=1)
prob = model.predict_proba(X, batch_size=1)

print('===[classified]===')
print('{0}'.format(Y == classes))
print('===[prob]=== ')
print('{0}'.format(prob))

===[classified]===
[[ True]
 [ True]
 [ True]
 [ True]]
===[prob]=== 
[[0.21298702]
 [0.9169551 ]
 [0.91943663]
 [0.99785703]]
